In [1]:
import torch
import torch.optim as optim
from tqdm import tqdm
from utils.bert_model import BiLSTM_CRF_BERT
from utils.utils import prepare, predict

from transformers import get_linear_schedule_with_warmup, AdamW
from transformers import AutoModel,BertTokenizerFast, BertTokenizer,BertModel

In [5]:
hparams = {
    'path':'/home/peitian_zhang/Data/NER/labeled_train.txt',
    'epochs': 50,
    'batch_size': 15,
    'embedding_dim': 768,
    'hidden_dim': 768,
    'device':'cuda:0',
    'bert':'ckiplab/albert-base-chinese',
}

In [3]:
tag2idx, vocab, loader = prepare(hparams)
hparams['vocab_size'] = len(vocab)
hparams['seq_length'] = loader.dataset.max_length

In [4]:
bert_model = BiLSTM_CRF_BERT(hparams,tag2idx).to(hparams['device'])

Some weights of AlbertModel were not initialized from the model checkpoint at ckiplab/albert-base-chinese and are newly initialized: ['albert.pooler.weight', 'albert.pooler.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
total_steps = len(loader) * hparams['epochs']

optimizer = optim.AdamW(bert_model.parameters(),lr=0.001)
# scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0, num_training_steps = total_steps)

for epoch in range(hparams['epochs']):
    tqdm_ = tqdm(enumerate(loader))
    total_loss = 0

    for step,x in tqdm_:
        bert_model.zero_grad()
        loss = bert_model.neg_log_likelihood(x)
        loss.backward()

        # prevent gradient explosion
        # torch.nn.utils.clip_grad_norm_(bert_model.parameters(), 1.0)

        optimizer.step()
        # scheduler.step()

        total_loss += loss.item()
        tqdm_.set_description("epoch {:d} , step {:d} , loss: {:.4f}".format(epoch+1, step, total_loss/(step+1)))

epoch 1 , step 64 , loss: -9085.8949: : 65it [00:47,  1.36it/s]
epoch 2 , step 64 , loss: -31231.9765: : 65it [00:47,  1.36it/s]
epoch 3 , step 64 , loss: -52123.6125: : 65it [00:46,  1.39it/s]
epoch 4 , step 64 , loss: -72145.6463: : 65it [00:47,  1.37it/s]
epoch 5 , step 64 , loss: -91053.4327: : 65it [00:47,  1.37it/s]
epoch 6 , step 64 , loss: -108740.0206: : 65it [00:48,  1.34it/s]
epoch 7 , step 64 , loss: -125400.5459: : 65it [00:48,  1.35it/s]
epoch 8 , step 64 , loss: -140919.7337: : 65it [00:47,  1.36it/s]
epoch 9 , step 64 , loss: -155171.3752: : 65it [00:47,  1.36it/s]
epoch 10 , step 64 , loss: -168184.3171: : 65it [00:47,  1.37it/s]
epoch 11 , step 64 , loss: -179987.4394: : 65it [00:47,  1.37it/s]
epoch 12 , step 64 , loss: -190501.6435: : 65it [00:48,  1.34it/s]
epoch 13 , step 64 , loss: -199911.0161: : 65it [00:47,  1.37it/s]
epoch 14 , step 64 , loss: -208174.5748: : 65it [00:46,  1.38it/s]
epoch 15 , step 64 , loss: -215236.1685: : 65it [00:48,  1.35it/s]
epoch 16 ,

In [6]:
with torch.no_grad():
    record = next(iter(loader))
    _, tag_seq = bert_model(record['token'], record['attn_mask'])
    print("Prediction:{}\n Ground Truth:{}".format(tag_seq, record['label']))

Prediction:tensor([[18,  3,  3,  ...,  0, 18,  3],
        [18,  3,  3,  ...,  0, 18,  3],
        [18,  3,  3,  ...,  0, 18,  3],
        ...,
        [18,  3,  3,  ...,  0, 18,  3],
        [18,  3,  3,  ...,  0, 18,  3],
        [18,  3,  3,  ...,  0, 18,  3]], device='cuda:0')
 Ground Truth:tensor([[ 3,  3,  3,  ...,  2,  2,  2],
        [ 3,  3,  3,  ...,  2,  2,  2],
        [14, 15, 15,  ...,  2,  2,  2],
        ...,
        [ 3,  3,  3,  ...,  2,  2,  2],
        [ 3,  3,  3,  ...,  2,  2,  2],
        [ 3,  3,  3,  ...,  2,  2,  2]])


below is still under development

In [ ]:
import torch
import torch.nn as nn
from transformers import BertModel,AutoModel

START_TAG = '<START>'
STOP_TAG = '<END>'
PAD_TAG = '<PAD>'

class BERT_NER(nn.Module):
    """
        fine-tune pretrained BERT-NER model on our dataset
    """

    def __init__(self, hparams, tag2idx):
        super().__init__()

        self.batch_size = hparams['batch_size']
        self.seq_length = hparams['seq_length']

        self.device = hparams['device']

        self.tag2idx = tag2idx
        self.tagset_size = len(tag2idx)
        self.idx2tag = {v:k for k,v in tag2idx.items()}

        self.lstm = nn.LSTM(self.embedding_dim, self.hidden_dim // 2, bidirectional=True)

        self.bert = AutoModel.from_pretrained(
            'ckiplab/albert-base-chinese-ner',
            # output hidden embedding of each transformer layer
        )

        self.hidden2tag = nn.Linear(self.embedding_dim, self.tagset_size)
        self.Softmax = nn.Softmax(dim=-1)
    
    def _bert_encode(token, attn_mask):
        output = self.bert(token, token_mask)
        embedding = output['last_hidden_state']
        tag_prob = self.Softmax(self.hidden2tag(embedding))

    
    def forward(x):


In [39]:
model = AutoModel.from_pretrained('ckiplab/albert-base-chinese', output_hidden_states=True)
# a = model(encoded['input_ids'], encoded['attention_mask'], labels=torch.tensor([[14,15,15,3,3,2,2,2,2,2]]))
a = model(encoded['input_ids'], encoded['attention_mask'])

Some weights of AlbertModel were not initialized from the model checkpoint at ckiplab/albert-base-chinese and are newly initialized: ['albert.pooler.weight', 'albert.pooler.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
